In [71]:
'''
Only look at "clueweb_FB15k_all-<lhs, rhs, rel, sent>.pkl" files
or perhaps the raw versions thereof. 

Investigate overlap of Clueweb textual mentions and FB15k triples
- How many unique textual mentions are associated with each triple
 	- with the most common triples (have the most examples)
 	- then investigate how paraphrastic these unique textual mentions are for 
 	  a given triple (triple is the label - distant supervision)
- which and how many unique text strings are associated with many triples?
	- if there are many, it confounds the assumption that text is 
	  representative of the triples they are mentions of

'''

import pickle, sys
sys.path.append('/Users/corbinrosset/Dropbox/Arora/QA-code/src/WordEmdgs/SIF/src/')
import data_io # eval
import random
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import pprint
from scipy.stats import iqr
import time
import random
import itertools
import gzip, cPickle
import matplotlib.pyplot as plt
from tsne import bh_sne
import mpld3
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import math

data_path = '/Users/corbinrosset/Dropbox/Arora/QA-data/FB15k_Clueweb/processed_text/'
execute_path = '/Users/corbinrosset/Dropbox/Arora/QA-code/src/process_clueweb/'
manifest_file = 'manifest.txt'
input_prefix = 'grouped_FB15k_clueweb_triples'
paraphrase_file = 'uncommon_triples_FB15k_clueweb_1.txt'

### notebook to analyze textual similarity of clueweb instances that share the same entity-relationship-entity triple
### uses the 5000 most common triples which are stored in 
### /Users/corbinrosset/Dropbox/Arora/QA-data/VanDurme_FB_annotations/annotated_clueweb/ClueWeb09_English_1/processed/grouped_textual_triples.pkl


In [53]:
###############################################################################
###                         	Globals                                     ###
###############################################################################

datatyp = 'all' # which .pkl files to load
NUM_FB1K_TRIPLES = 47291696
NUM_UNIQUE_SENTENCES = 6194853

entity2idx = None 
idx2entity = None 
num_entities_union_rels = 0 # sum of |Entities| + |Relatiionships| in FB15k
USE_FINAL = False #True
min_words_per_text = 2 ### min number of words needed to count a textual triple
num_triples = 5000 ### put top num_triples into the triples_to_extract below

unique_sent_map = {}
idx_2_sent_map = {}
text_per_triple_cntr = {} # count total number of textual instances f.e. triple
unique_text_per_triple = {} # set of unique text mentions f.e. triple


### the following is the word embeddings maps to use, download them pretrained
wordfiles = [#'../data/paragram_sl999_small.txt', # need to download it from John Wieting's github (https://github.com/jwieting/iclr2016)
    # '/Users/corbinrosset/Dropbox/GloVe/glove.840B.300d.txt'
    '/Users/corbinrosset/Dropbox/Paragrams/paragrams-XXL-SL999.txt'

    ]
weightfile = '/Users/corbinrosset/Dropbox/Arora/QA-code/src/WordEmdgs/SIF/auxiliary_data/enwiki_vocab_min200.txt'
weightparas = [1e-2] ### used 1e-2 #[-1,1e-1,1e-2,1e-3,1e-4] # (0.01 works best)
# weightparas = [-1,1e-1,1e-2,1e-3,1e-4] # (0.01 works best)

rmpcs = [0] #[0, 1, 2, 3, 4] # remove the first k principle components, (4 is best)
scoring_function = None
save_result = False 
 
###############################################################################
###                         Helper Classes                                  ###
###############################################################################

class params(object):
    
    def __init__(self):
        self.LW = 1e-5
        self.LC = 1e-5
        self.eta = 0.05

    def __str__(self):
        t = "LW", self.LW, ", LC", self.LC, ", eta", self.eta
        t = map(str, t)
        return ' '.join(t)


params = params()
parr4para = {}
sarr4para = {}
wordfile = wordfiles[0]

In [54]:
### load clueweb FB15k data
start = time.clock() 

text_per_triple_cntr = pickle.load(open(data_path + input_prefix + '-counts.pkl', 'r'))
unique_text_per_triple = pickle.load(open(data_path + input_prefix + '-text_sets.pkl', 'r'))
triple_per_text = pickle.load(open(data_path + input_prefix + '-triple_per_text_count.pkl', 'r'))
entity2idx = pickle.load(open(data_path + 'FB15k_entity2idx.pkl', 'r'))
idx2entity = pickle.load(open(data_path + 'FB15k_idx2entity.pkl', 'r'))
unique_sent_map = pickle.load(open(data_path + 'clueweb_FB15k_' + 'all-sent2idx.pkl', 'r'))
idx_2_sent_map = pickle.load(open(data_path + 'clueweb_FB15k_' + 'all-idx2sent.pkl', 'r'))
# num_entities_union_rels = np.max(entity2idx.values()) + 1
srtd_triples = sorted(text_per_triple_cntr, key=text_per_triple_cntr.__getitem__, reverse=True)

elapsed = time.clock()
elapsed = elapsed - start
print "loaded data in: ", elapsed
assert len(unique_text_per_triple.keys()) == len(text_per_triple_cntr.keys())
print 'Number of unique triples: ' + str(len(unique_text_per_triple.keys()))

loaded data in:  555.78672
Number of unique triples: 325337


In [4]:
### load word embeddings map
print 'loading word embeddings map...will take many minutes'
start = time.clock() 
(words, We) = data_io.getWordmap(wordfile)
elapsed = time.clock()
elapsed = elapsed - start
print 'word vectors loaded from ' + str(wordfile) + ' in ' + str(elapsed)
    

loading word embeddings map...will take many minutes


TypeError: not enough arguments for format string

In [5]:

###############################################################################
###            Helper Methods for the Original Textual Similarity Task      ###
###############################################################################

def evaluate_similarity(We, words, f, weight4ind, scoring_function, params):
    golds = []
    seq1 = []
    seq2 = []
    with open(f,'r') as fl:
        for i in fl:
            i = i.split("\t")
            p1 = i[0]; p2 = i[1]; score = float(i[2])
            X1, X2 = data_io.getSeqs(p1,p2,words)
            seq1.append(X1)
            seq2.append(X2)
            golds.append(score)
    x1,m1 = data_io.prepare_data(seq1)
    x2,m2 = data_io.prepare_data(seq2)
    m1 = data_io.seq2weight(x1, m1, weight4ind)
    m2 = data_io.seq2weight(x2, m2, weight4ind)
    scores = scoring_function(We,x1,x2,m1,m2, params)
    preds = np.squeeze(scores)
    return pearsonr(preds,golds)[0], spearmanr(preds,golds)[0]

def get_embeddings(We, words, data, weight4ind, params):

    ### asssuming data is the similarity dataset:
    seq, labels, sentences = [], [], []
    with open(data,'r') as fl:
        for idx, i in enumerate(fl):
            i = i.split("\t")
            p1 = i[0]; p2 = i[1]; score = float(i[2])
            X1, X2 = data_io.getSeqs(p1,p2,words)
            seq.append(X1)
            seq.append(X2)
            labels.append(score)
            labels.append(score)
            sentences.append(p1)
            sentences.append(p2)

    x1, w1 = data_io.prepare_data(seq)
    emb, embnorm = get_norm_embedding(We, x1, w1, params)
    emb = np.asarray(emb, dtype=np.float64)
    # perhaps divide each row by its L2 norm, emb /= embnorm
    print np.shape(emb)
    return emb, labels, sentences




###############################################################################
###                      SIF Averaging Methods                              ###
###############################################################################

def get_weighted_average(We, x, w):
    """
    Compute the weighted average vectors
    :param We: We[i,:] is the vector for word i
    :param x: x[i, :] are the indices of the words in sentence i
    :param w: w[i, :] are the weights for the words in sentence i
    :return: emb[i, :] are the weighted average vector for sentence i
    """
    n_samples = x.shape[0]
    emb = np.zeros((n_samples, We.shape[1]))
    for i in xrange(n_samples):
        emb[i,:] = w[i,:].dot(We[x[i,:],:]) / np.count_nonzero(w[i,:])
    return emb

def compute_pc(X,npc=1):
    """
    Compute the principal components
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: component_[i,:] is the i-th pc
    """
    svd = TruncatedSVD(n_components=npc, n_iter=7, random_state=0)
    svd.fit(X)
    return svd.components_

def remove_pc(X, npc=1):
    """
    Remove the projection on the principal components
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: XX[i, :] is the data point after removing its projection
    """
    pc = compute_pc(X, npc)
    if npc==1:
        XX = X - X.dot(pc.transpose()) * pc
    else:
        XX = X - X.dot(pc.transpose()).dot(pc)
    return XX

def get_norm_embedding(We, x1, w1, params):
    '''basically does what the scoring function below does, but for one sentence'''
    emb1 = get_weighted_average(We, x1, w1) # num samples by embedding dim
    if  params.rmpc > 0:
        emb1 = remove_pc(emb1, params.rmpc)
    emb1norm = np.sqrt((emb1 * emb1).sum(axis=1))
    return emb1, emb1norm

def weighted_average_sim_rmpc(We,x1,x2,w1,w2, params):
    """
    Compute the scores between pairs of sentences using weighted average + removing the projection on the first principal component
    :param We: We[i,:] is the vector for word i
    :param x1: x1[i, :] are the indices of the words in the first sentence in pair i
    :param x2: x2[i, :] are the indices of the words in the second sentence in pair i
    :param w1: w1[i, :] are the weights for the words in the first sentence in pair i
    :param w2: w2[i, :] are the weights for the words in the first sentence in pair i
    :param params.rmpc: if >0, remove the projections of the sentence embeddings to their first principal component
    :return: scores, scores[i] is the matching score of the pair i
    """
    emb1 = get_weighted_average(We, x1, w1)
    emb2 = get_weighted_average(We, x2, w2)
    if  params.rmpc > 0:
        emb1 = remove_pc(emb1, params.rmpc)
        emb2 = remove_pc(emb2, params.rmpc)

    inn = (emb1 * emb2).sum(axis=1) # inner products
    emb1norm = np.sqrt((emb1 * emb1).sum(axis=1))
    emb2norm = np.sqrt((emb2 * emb2).sum(axis=1))
    scores = inn / emb1norm / emb2norm # cosine similarity
    return scores







In [64]:
###############################################################################
###                 Helper Methods for Clueweb-related tasks                ###
###############################################################################
def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

### CLUEWEB
def get_statistics(lst):
    stats = {}
    stats['mean'] = np.mean(lst)
    stats['median'] = np.median(lst)
    # stats['quartiles'] = 
    stats['standard deviation'] = np.std(lst)
    stats['mean absolute deviation'] = np.mean(np.absolute(lst - np.mean(lst)))
    stats['interquartile range'] = iqr(lst)

    # plt.clera()
    # plt.hist(lst, bins=100)
    # plt.show()

    return stats

### CLUEWEB
def get_embeddings_clueweb(We, words, data, weight4ind, params, key):
    seq, labels, sentences = [], [], []
    key = str((idx2entity[key[0]], idx2entity[key[1]], idx2entity[key[2]]))
    for i, text_instance in enumerate(data):
        # print '\t\t' + str(text_instance) + ' ' + str(idx_2_sent_map[text_instance])
        ### remember text instances are tuples ("text goes here",)
        X = data_io.getSeq(idx_2_sent_map[text_instance], words)
        labels.append(key)
        seq.append(X)
        sentences.append(idx_2_sent_map[text_instance])

    x1, w1 = data_io.prepare_data(seq)
    emb, embnorm = get_norm_embedding(We, x1, w1, params)
    emb = np.asarray(emb, dtype=np.float64)
    # perhaps divide each row by its L2 norm, emb /= embnorm
    return emb, labels, sentences

### CLUEWEB
def evaluate_similarity_all_pairs(We, words, data, weight4ind, scoring_function, params, key):
    '''data here is a dictionary of 5k of the most commonly mentioned freebase triples 
        mapping to a list of their textual instances, which is at most 40k in length. 
        The goal here is to get a sense of how similar the instances in the list are with 
        each other - there are no labels here, just output say the average/median 
        similarity score of all pairs in the list, or a subsample of all pairs. 
    '''
    triple_sim_stats = {}
    avgs = []
    max_num_pairs = 10000
    counter = 10000
    seq1, seq2, sentcs = [], [], [] 
    
    if len(data) <= 0:
        print '\tnot enough data'
        return
    pairs = itertools.combinations(data, 2)
    numpairs = nCr(len(data), 2)
    print '\tnumber of possible pairs: ' + str(numpairs)
    

    if numpairs > max_num_pairs:
        prob = max_num_pairs/float(numpairs)
        for i, pair in enumerate(pairs):
            if random.random() > prob: ### subsample
                continue
            counter -= 1
            if counter <= 0:
                break
            pair = (idx_2_sent_map[pair[0]].strip(), idx_2_sent_map[pair[1]].strip())
            sentcs.append(pair)
            X1, X2 = data_io.getSeqs(pair[0], pair[1], words)
            seq1.append(X1)
            seq2.append(X2)
        print '\tlength of sampled sequence: ' + str(len(seq1))
    else:
        for i, pair in enumerate(pairs):
            pair = (idx_2_sent_map[pair[0]].strip(), idx_2_sent_map[pair[1]].strip())
            sentcs.append(pair)
            X1, X2 = data_io.getSeqs(pair[0], pair[1], words)
            seq1.append(X1)
            seq2.append(X2)
            
    assert len(seq1) == len(seq2)

    x1,m1 = data_io.prepare_data(seq1)
    x2,m2 = data_io.prepare_data(seq2)
    m1 = data_io.seq2weight(x1, m1, weight4ind)
    m2 = data_io.seq2weight(x2, m2, weight4ind)
    scores = scoring_function(We,x1,x2,m1,m2, params)
    preds = np.squeeze(scores)
    stats = get_statistics(preds)
    
    ### get top paraphrases
    idx = np.argsort(preds)
    preds = preds[idx]
    sentcs = np.array(sentcs)[idx]
    stats['top'] = [(i, j) for i, j in zip(preds[-50:], sentcs[-50:])]
    
    print 'done'
    return stats

In [72]:
# %matplotlib inline
# mpld3.enable_notebook()
mpld3.disable_notebook()

###############################################################################
###                 Manage and Visualize Clueweb Experiments                ###
###############################################################################

def t_sne(data):
    '''run_bh_tsne(data, no_dims=2, perplexity=50, theta=0.5, randseed=-1, verbose=False,initial_dims=50, use_pca=True, max_iter=1000):
        
        Run TSNE based on the Barnes-HT algorithm

        Parameters:
        ----------
        data: file or numpy.array
            The data used to run TSNE, one sample per row
        no_dims: int
        perplexity: int
        randseed: int
        theta: float
        initial_dims: int
        verbose: boolean
        use_pca: boolean
        max_iter: int
    '''

    X_2d = bh_sne(data, perplexity=30, theta=0.5) #, max_iter=600) #bh_sne(X, perplexity=50, theta=0.5)  
    return X_2d

def tooltip_style(text):
    return "<p style=\"color: #ffffff; background-color: #000000\">" + str(text) + "</p>"

def plot_clueweb(X, Y, popup_labels, title):
    '''X is a list of matrices of length n, each of which has its own class or 
    color, y. popup_labels is also a list of n lists, each containing the tag
    of individual points (rows in a matrix of X)'''

    ###############
    labels_flat = [item for sublist in Y for item in sublist]
    labels_set = set(labels_flat)
            
    fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'), figsize=(20,20))
    
    assert len(X) == len(Y)
    if popup_labels:
        assert len(popup_labels) == len(X)
        assert np.shape(X)[0] == np.shape(popup_labels)[0]
    ax.grid(color='white', linestyle='solid')
    colors = cm.rainbow(np.linspace(0,1, len(Y)))
    color_label_map = {}
    for c, label in zip(colors, labels_set):
        color_label_map[label] = c
#     print 'labels\' colors: ' + str(color_label_map)
        
    legend_partition, legend_strings = [], [] 
    for (x, group_labels, tag) in zip(X, Y, popup_labels):
        label = group_labels[0]
        assert all(label == i for i in group_labels)
        scatter = ax.scatter(x[:, 0], x[:, 1], c = color_label_map[label], alpha=0.3, label=label)
        legend_partition.append(scatter)
        legend_strings.append(label)
#         tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=[tooltip_style(t[0]) for t in tag])
        tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=[tooltip_style(t) for t in tag])
        mpld3.plugins.connect(fig, tooltip)
        
    l_keys, l_nums, recs = [], [], []
    print 'Legend: '
    for i, (key, value) in enumerate(color_label_map.items()):
        print str(i) + ' = ' + str(key)
        l_nums.append(str(i))
        l_keys.append(key)
        recs.append(mpatches.Rectangle((0,0),1,1,fc=value))

    plt.legend(recs, l_keys, loc='upper right', fontsize=8)
    if title:
        ax.set_title(title, size=12)
        
    mpld3.save_html(fig, title + '.html')
    mpld3.show()
#     mpld3.display() # doesn't seem to work for ipython notebooks?
    return

def clueweb_paraphrase_experiment():
    scoring_function = weighted_average_sim_rmpc
    with open(data_path + paraphrase_file) as g:
        ### only load textual mentions of triples with the most mentions
        triples = []
        for i in g.readlines():
            i = i.strip().split()
            triple = (entity2idx[i[0]], entity2idx[i[1]], entity2idx[i[2]])
            triples.append(triple)
            
        X, labels, sentences = [], [], []
        for t in triples:
            key = str((idx2entity[t[0]], idx2entity[t[1]], idx2entity[t[2]]))
            ### filter only meaningful and unique sentences that are long and not used by other triples
            d = [i for i in unique_text_per_triple[t] if (triple_per_text[i] <= 6 and len(idx_2_sent_map[i]) > 3)]
            print key + ' num valid texts: ' + str(len(d)) + ' out of num unique texts: ' \
                + str(len(unique_text_per_triple[t]))
            if len(d) <= 0:
                continue
            stats = evaluate_similarity_all_pairs(We, words, d, weight4ind, scoring_function, params, key)
            for top in stats['top']:
                print '\t\t' + str(np.round(top[0], 3)) + ' ' + str(top[1][0]) + ' <=> ' + str(top[1][1])
            print '\tmean: ' + str(stats['mean'])
            print '\tmediatn: ' + str(stats['median'])
            print '\tSTD: ' + str(stats['standard deviation'])
            print '\tMAD: ' + str(stats['mean absolute deviation']) 
            print '\tIQR: ' + str(stats['interquartile range'])
            
    
def clueweb_tsne_experiment():
     ### T-SNE on selected triples - A COUPLE OF EXPERIMENTS NEED TO BE DONE HERE!!!!
    max_exs_per_triple = 500
    with open(data_path + paraphrase_file) as g:
        ### only load textual mentions of triples with the most mentions
        triples = []
        for i in g.readlines():
            i = i.strip().split()
            triple = (entity2idx[i[0]], entity2idx[i[1]], entity2idx[i[2]])
            triples.append(triple)
            
        X, labels, sentences = [], [], []
        for t in triples:
            key = str((idx2entity[t[0]], idx2entity[t[1]], idx2entity[t[2]]))
            data_selected = [i for i in unique_text_per_triple[t] if triple_per_text[i] <= 6] ### a set of text instances
            print '\t' + key + ' num valid texts: ' + str(len(data_selected)) \
                + ' out of num unique texts: ' + str(len(unique_text_per_triple[t]))
            if len(data_selected) <= 0:
                continue
            x, labels_x, sentences_x = get_embeddings_clueweb(We, words, data_selected, weight4ind, params, t)
            # print '\tshape: ' + str(np.shape(x))
            if np.shape(x)[0] > max_exs_per_triple:  ### sample 200 instances to reduce runtime
                idx = np.random.choice(np.shape(x)[0], max_exs_per_triple)
                X.append(x[idx, :])
                labels.append((np.array(labels_x)[idx]).tolist())
                sentences.append((np.array(sentences_x)[idx]).tolist())
            else:
                X.append(x)
                labels.append(labels_x)
                sentences.append(sentences_x)
            print '\t' + str(np.shape(X[-1])) + ' ' + str((idx2entity[t[0]], idx2entity[t[1]], idx2entity[t[2]]))
        print 'done creating embeddings'
        X_stacked = np.asarray(np.vstack(tuple(X)), dtype=np.float64)
        X_2d = t_sne(X_stacked) #
        print 'done with tsne ' + str(np.shape(X_2d)) 

        ### must chunk the t-sne embeddings to match labels...
        X_2d_chunk = []
        size = 0
        for i in labels:
            X_2d_chunk.append(X_2d[size:(size + len(i))])
            size += len(i)

        for a, b, c in zip(X_2d_chunk, labels, sentences):
            assert len(a) == len(b) == len(c)
        return X_2d_chunk, labels, sentences


In [73]:
###############################################################################
###                                 Main                                    ###
###############################################################################


### tune weight and rmpc params in evaluation of similarity
for weightpara in weightparas:
    print '\n==================================\n'
    #### load weight for each word using idf or tf-idf
    word2weight = data_io.getWordWeight(weightfile, weightpara)
    print 'done with word2weight' 
    weight4ind = data_io.getWeight(words, word2weight)
    print 'done with weight4ind'
    
    for rmpc in rmpcs:
        print 'word vectors loaded from %s' % wordfile
        print 'word weights computed from %s using parameter a=%f' % (weightfile, weightpara)
        params.rmpc = rmpc
        print '*** remove the first %d principal components ***' % rmpc

        #################################################################
        ### modify the following lines for your purposes (clueweb or sim)
        #################################################################


        #################################################################
        #################################################################
        ### for text similarity task data:

        # for file in paraphrases_files:
        # scoring_function = weighted_average_sim_rmpc
        # triple_sim_stats, avgs = evaluate_similarity(We, words, data, weight4ind, scoring_function, params) ### only for clueweb

        # print 'average similarities per triple: ' + str(avgs)
        # print 'average of averages: ' + str(np.mean(avgs))
        # with open(data_dir + 'clueweb_triples_similarity_' + str(rmpc) + '_rmpc_' + str(weightpara) + '_weight.pkl', 'w') as f:
        #     pickle.dump(triple_sim_stats, f)

        #################################################################
        ### T-SNE
        # X, labels, sentences = get_embeddings(We, words, data, weight4ind, params)
        # t_sne(X, labels, sentences, 't-SNE on text similarity pairs')

        #################################################################
        #################################################################
        ### for clueweb data

        ### test similirity of textual instances of same FB triple...
        clueweb_paraphrase_experiment()

        #################################################################
#         X_2d_chunk, labels, sentences = clueweb_tsne_experiment()
#         plot_clueweb(X_2d_chunk, labels, sentences, title='t-SNE of 7 common FB15k-Clueweb textual instances [1]')
#         print 'done'
       
            
            



###############################################################################
###                                 Dead                                    ###
###############################################################################



done with word2weight
done with weight4ind
word vectors loaded from /Users/corbinrosset/Dropbox/Paragrams/paragrams-XXL-SL999.txt
word weights computed from /Users/corbinrosset/Dropbox/Arora/QA-code/src/WordEmdgs/SIF/auxiliary_data/enwiki_vocab_min200.txt using parameter a=0.010000
*** remove the first 0 principal components ***
('/m/0d19y2', '/medicine/disease/risk_factors', '/m/07jwr') num valid texts: 275 out of num unique texts: 323
	number of possible pairs: 37675
	length of sampled sequence: 9863
done
		0.716 virus hepatitis tuberculosis <=> aids tuberculosis
		0.718 viral hepatitis sexually transmitted diseases <=> virus hepatitis tuberculosis
		0.72 tuberculosis malaria activities including DOLLAR million global fund fight aids <=> services support global fund fight aids
		0.721 hepatitis typhoid fever cancers herpes pneumonia <=> epidemic prevention treatment infectious diseases malaria
		0.724 hepatitis infectious diseases <=> head lice hearing loss hearing test audiogram h